### Import packages

In [1]:
import sqlalchemy as sql
import pandas as pd

### Create sqlite engine

In [7]:
sql_file='sqlite_db_pythonsqlite.db'

In [8]:
engine=sql.create_engine('sqlite:///'+sql_file)

### Test connection

In [9]:
tables=engine.table_names()

In [10]:
print(tables)

['Bookings', 'Facilities', 'Members']


### Define function to query Country Club database

In [11]:
def QueryDB(engine,querystring):
    """Connect to SQL engine and return resulting query"""
    
    con=engine.connect()
    rs=con.execute(querystring)
    df=pd.DataFrame(rs.fetchall())
    df.columns=rs.keys()
    con.close()
    return df
    

In [12]:
test=QueryDB(engine,'SELECT * FROM Facilities')

In [13]:
test.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


### Question 10

In [14]:
q10='''
SELECT *
FROM (
	SELECT 
		f.facid,
		f.name,
		SUM(CASE WHEN b.memid=0 THEN f.guestcost
    	    ELSE f.membercost END) AS revenue
	FROM Bookings as b
	LEFT JOIN Facilities as f
	ON f.facid=b.facid
	GROUP BY f.name
	ORDER BY revenue DESC) as sub
WHERE sub.revenue <1000
'''

In [17]:
df=QueryDB(engine,q10)
df.head()

,facid,name,revenue
0,2,Badminton Court,604.5
1,8,Pool Table,265.0
2,7,Snooker Table,115.0
3,3,Table Tennis,90.0


### Question 11

In [51]:
q11='''
SELECT 
	m1.surname||', '||m1.firstname AS member,
	m2.surname||', '||m2.firstname AS reference
FROM Members as m1
INNER JOIN Members as m2
ON m1.recommendedby=m2.memid
WHERE m1.memid!=0
ORDER BY member
'''

In [53]:
df=QueryDB(engine,q11)
df.head(10)

,member,reference
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


### Question 12

In [54]:
q12="""
SELECT sub.name,
	COUNT(sub.name) AS member_usage
FROM (
	SELECT 
    	b.memid,
		f.name
	FROM Bookings AS b
	LEFT JOIN Facilities as f
	ON b.facid=f.facid
	WHERE b.memid!=0 ) AS sub
GROUP BY sub.name
ORDER BY member_usage DESC
"""

In [56]:
df=QueryDB(engine,q12)
df.head(10)

,name,member_usage
0,Pool Table,783
1,Snooker Table,421
2,Massage Room 1,421
3,Table Tennis,385
4,Badminton Court,344
5,Tennis Court 1,308
6,Tennis Court 2,276
7,Squash Court,195
8,Massage Room 2,27


### Question 13

In [71]:
q13='''
SELECT sub.name,
	sub.month,
	COUNT(sub.name) AS member_usage
FROM (
	SELECT 
    	b.memid,
		f.name,
    	STRFTIME('%m', b.starttime) AS month
	FROM Bookings AS b
	LEFT JOIN Facilities as f
	ON b.facid=f.facid
	WHERE b.memid!=0 ) AS sub
GROUP BY sub.name,sub.month
ORDER BY sub.name,sub.month
'''

In [73]:
df=QueryDB(engine,q13)
df.head(20)

,name,month,member_usage
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103
